In [1]:
import pandas as pd
import re
from underthesea import text_normalize, word_tokenize

In [2]:
df_train = pd.read_excel("./train_nor_811.xlsx")
df_dev   = pd.read_excel("./valid_nor_811.xlsx")
df_test  = pd.read_excel("./test_nor_811.xlsx")


In [3]:
for df in [df_train, df_dev, df_test]:
    if 'Sentence' in df.columns:
        df.rename(columns={'Sentence': 'content'}, inplace=True)

In [4]:
df_misspell = pd.read_csv('./vietnamese-misspell.csv')
misspell_dict = dict(zip(df_misspell['wrong'], df_misspell['right']))

In [5]:
emoticon_dict = {
    ":)": "☺️",
    ":))": "☺️",
    ":)))": "☺️",
    "=))": "😄",
    "=)": "😄",
    ":D": "😀",
    ":DD": "😁",
    ":<": "😞",
    ":(": "☹️",
    ":'(": "😢",
    ":'(": "😢",
    ":')": "😂",
    ":')": "😂",
    "<3": "❤️",
    "</3": "💔",
    ":P": "😛",
    ":p": "😛",
    ":O": "😲",
    ":o": "😲",
    ";)": "😉",
    ";-)": "😉",
    ":3": "😺",
    ":^)": "😊",
    "^_^": "😊",
    "-_-": "😑",
    ">_<": "😣",
    "XD": "😆",
    "xD": "😆",
    "T_T": "😭",
    ";_;": "😭",
    ":|": "😐",
    ":/": "😕",
    ":-/": "😕",
    ":-\\": "😕",
    ":'D": "😆",
    ":'D": "😆",
    ":-*": "😘",
    ":*": "😘",
    "<<": "😓",
    ">_>": "😒",
    "<_<": "😒",
    "^\\^": "😆",
    "\\^_^/": "🎉",
    "*^_^*": "🎉",
    "\\o/": "🙌",
    "\\O/": "🙌",
    "O_O": "😳",
    "o_o": "😳",
    ">:O": "😠",
    "^^": "☺️"
}


In [6]:
def standardize_emoticon(text):
    # Gộp các chuỗi emoticon thường gặp về chuẩn (gộp lặp, ví dụ ^^ ^^ ^^ -> ^^)
    # Gộp emoticon kiểu :) về 1
    text = re.sub(r'((:\)+))', ':)', text)
    text = re.sub(r'((=\)+))', '=)', text)
    text = re.sub(r'((\^_?\^)+)', '^^', text)        # ^^, ^_^, ^^ ^^, ...
    text = re.sub(r'(<3+)', '<3', text)
    text = re.sub(r'(\)+)', ')', text)
    text = re.sub(r'(\(+)', '(', text)
    # Gộp mọi chuỗi ^^ liên tiếp về 1 ^^ (kể cả có cách ra)
    text = re.sub(r'(\^\^)(\s+\^\^)+', '^^', text)
    # Gộp ((
    text = re.sub(r'(\(+)', '(', text)
    text = re.sub(r'(\)+)', ')', text)
    return text

In [7]:
def standardize_emoticon(text):
    # Gộp các chuỗi emoticon thường gặp về chuẩn (gộp lặp, ví dụ ^^ ^^ ^^ -> ^^)
    # Gộp emoticon kiểu :) về 1
    text = re.sub(r'((:\)+))', ':)', text)
    text = re.sub(r'((=\)+))', '=)', text)
    text = re.sub(r'((\^_?\^)+)', '^^', text)        # ^^, ^_^, ^^ ^^, ...
    text = re.sub(r'(<3+)', '<3', text)
    text = re.sub(r'(\)+)', ')', text)
    text = re.sub(r'(\(+)', '(', text)
    # Gộp mọi chuỗi ^^ liên tiếp về 1 ^^ (kể cả có cách ra)
    text = re.sub(r'(\^\^)(\s+\^\^)+', '^^', text)
    # Gộp ((
    text = re.sub(r'(\(+)', '(', text)
    text = re.sub(r'(\)+)', ')', text)
    return text

In [8]:
def convert_emoticon(text, emoticon_dict):
    # Duyệt emoticon dài trước
    for emo in sorted(emoticon_dict, key=len, reverse=True):
        # Chỉ thay thế khi là nguyên một cụm (dùng word boundary nếu cần)
        text = re.sub(re.escape(emo) + r'(?=\s|$)', emoticon_dict[emo], text)
    return text

In [9]:
def standardize_word(text, misspell_dict):
    # Gộp ký tự kéo dài (đẹpppp -> đẹp)
    text = re.sub(r'(\w)\1{2,}', r'\1', text)
    # Chuẩn hóa chính tả/viết tắt
    for wrong, right in misspell_dict.items():
        text = re.sub(r'\b' + re.escape(wrong) + r'\b', right, text)
    return text

In [10]:
teencode_map = {}
with open("./vietnamese-teencode.txt", "r", encoding="utf8") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 2:
            teencode_map[parts[0]] = parts[1]

In [11]:
import re

def remove_duplicate_emoji(text):
    # Xóa emoji trùng lặp liên tiếp (vd: 😄😄😄 -> 😄)
    emoji_pattern = re.compile(
        r'([\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF])\1+'
    )
    return emoji_pattern.sub(r'\1', text)


In [12]:
def preprocess_text(text, emoticon_dict, misspell_dict):
    text = str(text)
    text = standardize_emoticon(text)
    text = convert_emoticon(text, emoticon_dict)
    #text = remove_duplicate_emoji(text)
    for abbr, full in teencode_map.items():
        pattern = rf"\b{re.escape(abbr)}\b"
        text = re.sub(pattern, full, text, flags=re.IGNORECASE)
    text = standardize_word(text, misspell_dict)
    return text

In [13]:
text = "ước gì sau này về già vẫn có thể như cụ này :)) :)))) :)))"
text = preprocess_text(text, emoticon_dict, misspell_dict)
print(text) 

ước gì sau này về già vẫn có thể như cụ này ☺️ ☺️ ☺️


In [14]:
# Ví dụ 1: Emoticon lặp nhiều lần
text1 = "ước gì sau này về già cũng có thể như cụ này :)) :)))) :)))"
print("Input 1:", text1)
print("Output 1:", preprocess_text(text1, emoticon_dict, misspell_dict))
print()

# Ví dụ 2: Lỗi chính tả và viết tắt
text2 = "Đẹppppp quáaaa, m.n chuc mng nămmm mơi nhéee =)) <33"
print("Input 2:", text2)
print("Output 2:", preprocess_text(text2, emoticon_dict, misspell_dict))
print()

# Ví dụ 3: Emoticon khác nhau + emoji unicode lặp lại
text3 = "Tuyệt vời quá  ^^ ^^ ^^ 😍😍😍"
print("Input 3:", text3)
print("Output 3:", preprocess_text(text3, emoticon_dict, misspell_dict))
print()

# Ví dụ 4: Có lỗi đánh máy và nhiều emoticon tiêu cực
text4 = "tui buồn lắm :((( :((( hicccccc"
print("Input 4:", text4)
print("Output 4:", preprocess_text(text4, emoticon_dict, misspell_dict))
print()

# Ví dụ 5: Trái tim, emoticon và các ký tự lặp
text5 = "iu quá <33 <3333 <333"
print("Input 5:", text5)
print("Output 5:", preprocess_text(text5, emoticon_dict, misspell_dict))
print()




Input 1: ước gì sau này về già cũng có thể như cụ này :)) :)))) :)))
Output 1: ước gì sau này về già z có thể như cụ này ☺️ ☺️ ☺️

Input 2: Đẹppppp quáaaa, m.n chuc mng nămmm mơi nhéee =)) <33
Output 2: Đẹp quáa, m.n chuc mọi năm mơi nhéee 😄 ❤️

Input 3: Tuyệt vời quá  ^^ ^^ ^^ 😍😍😍
Output 3: Tuyệt vời quá  ☺️ 😍😍😍

Input 4: tui buồn lắm :((( :((( hicccccc
Output 4: tui buồn lắm ☹️ ☹️ hic

Input 5: iu quá <33 <3333 <333
Output 5: yêu quá ❤️ ❤️ ❤️



In [15]:
test_cases = [
    "ước gì sau này về già vẫn có thể như cụ này :)) :)))) :)))",
    "Tuyệt vời quá ^^ ^^ ^^ 😍😍😍",
    "đẹppp quáaaa =))) :((((( ^^ :)) ^^",
    "qúy hóa quá <33 ^^ ^^ ^^",
    "ngủ ngoan nha :3 :3 :3 =))))",
    "Đcm tụi m ngu quá =)))))))))",
    "T đ hiểu, sao tụi m làm được vậy luôn á??",
    "Thằng này đc :))))))))))))))))"
]

for idx, text in enumerate(test_cases, 1):
    print(f"Input {idx}: {text}")
    print("Output:", preprocess_text(text, emoticon_dict, misspell_dict))
    print()


Input 1: ước gì sau này về già vẫn có thể như cụ này :)) :)))) :)))
Output: ước gì sau này về già vẫn có thể như cụ này ☺️ ☺️ ☺️

Input 2: Tuyệt vời quá ^^ ^^ ^^ 😍😍😍
Output: Tuyệt vời quá ☺️ 😍😍😍

Input 3: đẹppp quáaaa =))) :((((( ^^ :)) ^^
Output: đẹp quáa 😄 ☹️ ☺️ ☺️ ☺️

Input 4: qúy hóa quá <33 ^^ ^^ ^^
Output: qúy hóa quá ❤️ ☺️

Input 5: ngủ ngoan nha :3 :3 :3 =))))
Output: ngủ ngoan nha 😺 😺 😺 😄

Input 6: Đcm tụi m ngu quá =)))))))))
Output: Đcm tụi m ngu quá 😄

Input 7: T đ hiểu, sao tụi m làm được vậy luôn á??
Output: T đ hiểu, sao tụi m làm được vậy luôn á??

Input 8: Thằng này đc :))))))))))))))))
Output: Thằng này được ☺️



In [16]:
df_test['content_clean'] = df_test['content'].apply(lambda x: preprocess_text(x, emoticon_dict, misspell_dict))
df_test[['content', 'content_clean']].head(10)

,content,content_clean
0,người ta có bạn bè nhìn vui thật,người ta có bạn bè nhìn vui thật
1,cho nghỉ viêc mói đúng sao goi là kỷ luật,cho nghỉ viêc mói đúng sao gọi là kỷ luật
2,kinh vãi 😡,kinh vãi 😡
3,nhà thì không xa lắm nhưng chưa bao giờ đi vì ...,nhà thì không xa lắm nhưng chưa bao giờ đi vì ...
4,bố không thích nộp đấy mày thích ý kiến không,bố không thích nộp đấy mày thích ý kiến không
5,bắt cóc bỏ đĩa . quanh đi quẩn lại chỉ khổ dân...,bắt cóc bỏ đĩa . quanh đi quẩn lại chỉ khổ dân...
6,per khi nào bạn có bằng xe ôtô thì bạn mới hiể...,per khi nào bạn có bằng xe ôtô thì bạn mới hiể...
7,một hành động đẹp đầy tính nhân văn ☺️☺️,một hành động đẹp đầy tính nhân văn ☺️☺️
8,anh nhỏ nhẹ xin 100k rồi trước khi đi còn chào...,anh nhỏ nhẹ xin 100k rồi trước khi đi còn chào...
9,per trình mày vẫn còn thấp chán =)),per trình mày vẫn còn thấp chán 😄


In [17]:
df_train['content_clean'] = df_train['content'].apply(lambda x: preprocess_text(x, emoticon_dict, misspell_dict))
df_train[['content', 'content_clean']].head(10)

,content,content_clean
0,cho mình xin bài nhạc tên là gì với ạ,cho mình xin bài nhạc tên là gì với ạ
1,cho đáng đời con quỷ . về nhà lôi con nhà mày ...,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,lo học đi . yêu đương lol gì hay lại thích học...,lo học đi . yêu đương lồn gì hay lại thích học...
3,uớc gì sau này về già vẫn có thể như cụ này :)),uớc gì sau này về già vẫn có thể như cụ này ☺️
4,mỗi lần có video của con là cứ coi đi coi lại ...,mỗi lần có video của con là cứ coi đi coi lại ...
5,thằng kia sao mày bắt vợ với bồ tao dọn thế ki...,thằng kia sao mày bắt vợ với bồ tao dọn thế ki...
6,một lí do trog muôn vàn lí do,một lí do trong muôn vàn lí do
7,thật hay đùa ác vậy . không thể tin được,thật hay đùa ác vậy . không thể tin được
8,"ko phải con mình , mà xem còn thấy đau như vậy...","không phải con mình , mà xem còn thấy đau như ..."
9,per nghe đi rồi khóc 1 trận cho thoải mái . đừ...,per nghe đi rồi khóc 1 trận cho thoải mái . đừ...


In [18]:
df_dev['content_clean'] = df_dev['content'].apply(lambda x: preprocess_text(x, emoticon_dict, misspell_dict))
df_dev[['content', 'content_clean']].head(10)

,content,content_clean
0,"tính tao tao biết , chẳng có chuyện gì có thể ...","tính tao tao biết , chẳng có chuyện gì có thể ..."
1,"lại là lào cai , tự hào quê mình quá :))","lại là lào cai , tự hào quê mình quá ☺️"
2,bị từ chối rồi,bị từ chối rồi
3,tam đảo trời đẹp các mem à,tam đảo trời đẹp các mem à
4,đọc bình luận của thằng đó không thiếu chữ nào 😂😂,đọc bình luận của thằng đó không thiếu chữ nào 😂😂
5,crush tao vẫn còn trinh nhé 😘,crush tao vẫn còn trinh nhé 😘
6,nó nói không đúg ư . đg của nhà bà ư . nó đóg ...,nó nói không đúng ư . đang của nhà bà ư . nó đ...
7,gap kiểu này chắc đái ra quần quá,gặp kiểu này chắc đái ra quần quá
8,yêu em mọa mọa 😘😘,yêu em mọa mọa 😘😘
9,có ai như tao không đọc đề thì quen nhưng lại ...,có ai như tao không đọc đề thì quen nhưng lại ...


In [19]:
df_train.to_csv("./df_train_clean.csv", index=False)
df_dev  .to_csv("./df_dev_clean.csv",   index=False)
df_test .to_csv("./df_test_clean.csv",  index=False)
print("Cleaned files written to /content/")

Cleaned files written to /content/
